# Note:  Run all cells in oder to excecute the program

# Project Description:
## __Tittle:__ Airport Data Management and Visualization
This repository contains a comprehensive GUI-based application tailored for the management and visualization of airport data. The application showcases the integration of a MongoDB database, an intuitive user interface, and backend processing capabilities.

## Key Features:

__Data Management:__ 
Load, filter, and interact with airport-related data, including details about runways, frequencies, and more.

__Visualization:__ 
Generate graphical representations for statistical insights, ranging from mean, median, mode statistics to correlation studies.

__Interactive User Interface:__ 
Constructed using the tkinter library, the UI is user-friendly, making data access and interaction seamless.

__Backend Processing:__ 
Integrated with the pandas library for efficient data manipulation, handling, and statistical computations.

__Database Integration:__ 
Utilizes the pymongo API to interact with a MongoDB database, demonstrating competency in database management and data retrieval.

## Technologies Used:

-  Python: The core programming language.
-  Tkinter: For building the GUI.
-  MongoDB: Database used for storing airport data.
-  PyMongo: Python API used to connect with MongoDB.
-  Pandas: For data manipulation and analysis.


# Imports & Initial Configurations:

In [ ]:
""" imports & settings """

import tkinter as tk
from tkinter import *
from tkinter import ttk
from tkinter import filedialog
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import style
from matplotlib import cm
import pymongo
import json
import csv
import time


# Initialize Database Connection:

This section establishes a connection to the MongoDB server running on the local machine.  
For this code to function properly, you'll need to have MongoDB server installed and running locally.  
To install MongoDB and set it up on your machine, you can refer to the official MongoDB documentation:  
[Official MongoDB Installation Guide](https://docs.mongodb.com/manual/installation/)

Once MongoDB is installed and running, this code connects to a database named 'summative' and   
to three collections within that database: 'airports_collection', 'runways_collection', and   
'frequencies_collection'.

In [ ]:
""" Initialize Database Connection: """

client = pymongo.MongoClient("mongodb://localhost:27017/") # Initialize mongoDB

db = client['summative']
collection_airports = db['airports_collection']
collection_runways = db['runways_collection']
collection_frequencies = db['frequencies_collection']



__Drop (Flush) Database Collections:__

This section contains commented-out code that, when executed, will drop or delete the associated 
database collections. This functionality is useful during the development and testing phase, 
allowing for a fresh start by removing existing data in the database.

__WARNING:__

If you're trying to test the database functionalities of the UI, especially the CSV file loading,
you might need to execute this section to flush out existing data in the database.

However, use this with caution! Executing this will permanently delete the data in the specified collections.

In [ ]:
"""
Uncomment the lines below to drop the respective collections:
"""
# db.frequencies_collection.drop()
# db.airports_collection.drop()
# db.runways_collection.drop()
"""
Note:
This cell was primarily used during the development phase to clear the database collections and perform tests.
"""

# Database Utilities

## DataBase Functions Description

The provided code contains a set of functions specifically designed to handle and manipulate data. Below is a brief description of each:

1. **read_csv()**: 
    - Reads a CSV file through a file dialog.
    - Validates the file type.
    - Identifies the type of dataset by checking the last column name.
    - Provides a preview of the selected CSV.

2. **clear_tree()**: 
    - Clears the existing TreeView, which is used for data display.

3. **to_mongoDB()**: 
    - Converts the selected CSV file data to MongoDB.
    - Validates the data and handles potential duplicate entries or unexpected errors.
    - Updates the database and reloads the displayed data.

4. **check_progress(count, lenght)**: 
    - Monitors the progress of data writing into the DB.
    - Provides user feedback on progress in 10% increments.

5. **csv_to_json(path_csv, key_name)**: 
    - Converts the given CSV to a temporary JSON file.
    - The function accepts a CSV path and a key name which becomes the root node of the JSON.

The code ensures efficient handling of data by validating file types, offering previews, and providing user feedback. It also safeguards the database from duplicate entries.


In [ ]:
""" functions DataBase """

# This func will find be used to find the csv file and generate a preview.
def read_csv():

    global csv_dataFrame
    global label_csv
    global path_csv
    global db_type
    global file_loaded_status

    # Specifies the types of files to be opened
    filetypes = (('CSV files', '*.csv'), ('All files', '*.*'))

    # file opening window and store path
    path_csv = filedialog.askopenfilename(title='Open a CSV File', filetypes = filetypes)

    # This if statement Splits the path string and check extension to verify is a csv (made lower case)
    if path_csv.split('.')[1].lower() != 'csv': 

        label_csv = tk.Label(label_button_csv, text='Not a CSV file, try again... ', 
                                    fg='red').place(x=x_coord , y=y_coord+30)
        display_label_db('')
        # we are using a variable that stores the status of the fila path load.
        # This var is initialized as true by the time the GUI frame is created, it is
        # changed to false if the file selected is not a csv.
        file_loaded_status = False

    else:
        
        # Initialize variable type wil null value.
        db_type = None
        # Initialize variable load status var.
        file_loaded_status = True
        # if the var status is correct, we then create a dataframe from the csv file selected.
        csv_dataFrame = pd.read_csv(path_csv)

        # These if conditional statements checks what kind of dataset we are dealing with
        # and sets a a variable status:
        # The var will have a unique character identifying whcih type of dataset we have loaded.
        # In order to identify the sets, I have scanned the last column name from each set as they are 
        # unique. We assume all datasets will have same column names as specified in the brief.
        # ****** There could be a possibility that user loads a different kind of dataset that by coincidende
        # it cobtains a last column with a name that matches our conditional statements *******
        # We could expands the checks and verify that every single column matches for example. 

        if (csv_dataFrame.columns[-1]) == 'frequency_mhz':
            db_type = 'f'

        elif (csv_dataFrame.columns[-1]) == 'keywords':
            db_type = 'a'

        elif (csv_dataFrame.columns[-1]) == 'he_displaced_threshold_ft':
            db_type = 'r'



        # feedback to user
        label_csv = tk.Label(label_button_csv, text='CSV file succesfuly loaded', fg='green').place(x=x_coord , y=y_coord+30)
        # feedback to user
        display_label_db('')
        # clear old tree view
        clear_tree()

        # The remaining code below generates a table preview of the selected csv file
        # setup new tree view
        my_tree['column'] = list(csv_dataFrame.columns)
        my_tree['show'] = 'headings'

        # tree headers
        for column in my_tree['column']:
            my_tree.column(column,width=60, stretch=0)
            my_tree.heading(column, text=column)
            # my_tree.config(selectmode='extended')
            

        # tree data
        df_rows = csv_dataFrame.to_numpy().tolist()
        for row in df_rows:
            my_tree.insert('','end', values=row)


        my_tree.place(x=0, y=0)

        # setup scrollbars for the yable preview:
        treescrollx = tk.Scrollbar(frame1, orient='horizontal', command=my_tree.xview)
        my_tree.config(xscrollcommand=treescrollx.set)
        treescrollx.place(x=10, y=720, width=1210)
        treescrollx.config(command=my_tree.xview)

        treescrolly = tk.Scrollbar(frame1, orient='vertical', command=my_tree.yview)
        my_tree.config(yscrollcommand=treescrolly.set)
        treescrolly.place(x=1222, y=170, height=550)
        treescrolly.config(command=my_tree.yview)

# function to clear element from the treeview
def clear_tree():

    my_tree.delete(*my_tree.get_children())

    

# function wwil insert the data from the csv file into the mongo DB.
# the func is able to detect the kind of data set by analazing the name of the very last column
# (they are all different).
def to_mongoDB():

    # initialize variable that stores status of database update
    db_add_status = 'pass'  

    # Check if a file has benn properly loaded before attemting to convert from csv to json,
    # if not, disply feedback to user. 
    if file_loaded_status == True:

        
        display_label_db('Adding data... Please wait...')

        # Conditionl statements that check which type of dataset has been loaded by checking
        # the previously set variable db_type 
        if db_type == 'r':  #runways

            display_label_db('Adding runways data to the DB...')
            time.sleep(1)  # adds a little time for displaying message
            
            collection = db['runways_collection'] # sets the db collection accordingly 

            display_label_db('Runways DB Collection opened...')

            # restructure data from csv to json, this will create a temp json file in the relative path
            csv_to_json(path_csv, 'runways') 
            path_json = 'json_temp_data.json'

            # send json file into the db
            with open(path_json) as j:

                display_label_db('Writting json into DB collection... Please wait...')
                data = json.load(j)

                # these3 lines of code will set variables used to display progress in steps of 10%
                # each data set has difefrent sizes so we check each and set vars accordingly before
                # passing them to the fucntion that disply progress check_progress()  
                lenght = len(data['runways'])

                count = 1


                # iterates over json file and instert each entry inti the db.
                for values in data['runways']:

                    # the original id values are kept and used in the db entries.
                    values['_id'] = values['id']  

                    # try catch set in order to handle the DuplicateKeyError.
                    # the app prototype can only add unique Ids at this moment.
                    try:
                        collection.insert_one(values)
                        count += 1 # counter used for the purpose of progess displsy
                    except pymongo.errors.DuplicateKeyError:
                        # this variable status is being used to disply appropiate feedback messages 
                        db_add_status = 'fail' 
                        break
                    except:
                        db_add_status = 'unexpected'

                    # we send info to progress function every completed iteration
                    check_progress(count, lenght)
                
            # this section ensures the appropiate message disply by checking var status
            if db_add_status == 'pass':
                display_label_db('Runways Data successfully added to database!','green')
            elif db_add_status == 'fail':
                display_label_db('New data contains duplicated IDs.\n'
                                        +'For the moment, this prototype can only add data with unique IDs.\n'+
                                        'Please check the dataset and try again', 'red')
            elif db_add_status == 'unexpected':
                display_label_db('Something went wrong','red')

            # after this whole process is done, we check the status of the db and reload data into app 
            check_db_status()
            load_db_data()
            
            
        # Same comments as previous if statement 
        elif db_type == 'a': #airports
            
            display_label_db('Adding airports data to the DB...')
            time.sleep(1)

            collection = db['airports_collection']

            display_label_db('Airports DB Collection opened...')

            csv_to_json(path_csv, 'airports')

            path_json = 'json_temp_data.json'



            with open(path_json) as j:


                display_label_db('Writting json into DB collection... Please wait...')
                data = json.load(j)
                

                lenght = len(data['airports'])

                count = 1

                for values in data['airports']:
                    values['_id'] = values['id']  # Do I still need this???? maybe not.
                    try:

                        collection.insert_one(values)
                        count += 1

                    except pymongo.errors.DuplicateKeyError:
                        db_add_status = 'fail'
                        break
                    except:
                        db_add_status = 'unexpected'


                    check_progress(count, lenght)

            if db_add_status == 'pass':
                display_label_db('Airports Data successfully added to database!','green')
            elif db_add_status == 'fail':
                display_label_db('New data contains duplicated IDs.\n'
                                        +'For the moment, this prototype can only add data with unique IDs.\n'+
                                        'Please check the dataset and try again', 'red')
            elif db_add_status == 'unexpected':
                display_label_db('Something went wrong','red')


            display_info_bottom('')
            check_db_status()
            load_db_data()

        # Same comments as previous if statement 
        elif db_type == 'f': #freqs  


            display_label_db('Adding frequencies data to the DB...')
            time.sleep(1)


            collection = db['frequencies_collection']

            display_label_db('Frequencies DB Collection opened...')
        
            csv_to_json(path_csv, 'frequencies')

            display_label_db('Temp json file successfully created...')

            path_json = 'json_temp_data.json'

            display_label_db('Assigning path to temp json...')



            with open(path_json) as j:

                display_label_db('Writting json into DB collection... Please wait...')

                data = json.load(j)

                lenght = len(data['frequencies'])

                count = 1

                for values in data['frequencies']:

                    values['_id'] = values['id']  # Do I still need this???? maybe not.
                    try:
                        collection.insert_one(values)
                        count += 1
                    except pymongo.errors.DuplicateKeyError:
                        db_add_status = 'fail'
                        break
                    except:
                        db_add_status = 'unexpected'                    
                    

                    check_progress(count, lenght)

            if db_add_status == 'pass':
                display_label_db('Frequencies Data successfully added to database!','green')
            elif db_add_status == 'fail':
                display_label_db('New data contains duplicated IDs.\n'
                                        +'For the moment, this prototype can only add data with unique IDs.\n'+
                                        'Please check the dataset and try again', 'red')
            elif db_add_status == 'unexpected':
                display_label_db('Something went wrong','red')

            check_db_status()
            load_db_data()
        
        # If db_type variable didn't match any of the conditions above means the var was not set although
        # a csv file was loaded. This means that the csv file loaded does not have the same expected
        # column structure.
        # display a message if the user attempts to load a different kind of csv dataset 
        else:
            display_label_db('Dataset columns do not match the expected structure, please verify','red')
        

    else: display_label_db('Please select a csv file before adding to DataBase','red')


def check_progress(count, lenght):

    tenth = int(lenght/10)

    if count == tenth:
        display_label_db('Writting json into DB collection... Progress 10%... ')

    elif count == (tenth*2):
        display_label_db('Writting json into DB collection... Progress 20%... ')

    elif count == (tenth*3):
        display_label_db('Writting json into DB collection... Progress 30%... ')

    elif count == (tenth*4):
        display_label_db('Writting json into DB collection... Progress 40%... ')

    elif count == (tenth*5):
        display_label_db('Writting json into DB collection... Progress 50%... ')

    elif count == (tenth*6):
        display_label_db('Writting json into DB collection... Progress 60%... ')

    elif count == (tenth*7):
        display_label_db('Writting json into DB collection... Progress 70%... ')

    elif count == (tenth*8):
        display_label_db('Writting json into DB collection... Progress 80%... ')

    elif count == (tenth*9):
        display_label_db('Writting json into DB collection... Progress 90%... ')

        

# This function converts the csv file table into a json file structure adding a json rootnode:
# The funtion takes 2 argumets, a file path and the key name for the json file.
# Pass paths and a name for the json root node.

def csv_to_json(path_csv, key_name): 

    display_label_db('Converting temporary json file... Please wait...')

    path_json = 'json_temp_data.json'
    temp_list = []
    temp_dict = {}

    # open csv file and convert to a dict structure
    with open(path_csv) as f:
        csv_f = csv.DictReader(f)

        display_label_db('Converting csv to json...')

        # iterate over dict elements and append to our new customized temp list
        for row in csv_f:

            temp_list.append(row)

        display_label_db('Csv to json converted...')

    # create a temp dict setting the json root node that was passed as function argument
    temp_dict[key_name] = temp_list
    display_label_db('Temp dict created...')

    # now we dump the temp dict structure into a temp json file
    with open(path_json, 'w') as j:

        json.dump(temp_dict, j)



# Display Utilities 
## Functions for Displaying Information

This suite of functions is designed to dynamically update various label widgets in the GUI to display relevant information or feedback to the user:

1. **display_label_db(text, fg='black')**:
    - Updates the text and foreground color of the `label_db` widget.
    - Parameters:
        * `text`: Text to be displayed.
        * `fg`: Foreground color (default is black).

2. **display_label_graphs(text)**:
    - Sets the text of the `labelFrame_graph` widget.
    - Parameter:
        * `text`: Text to be displayed.

3. **display_info_bottom(text, fg='black')**:
    - Clears any existing graphics and updates the `label_graph_bottom` widget's text and foreground color.
    - Parameters:
        * `text`: Text to be displayed.
        * `fg`: Foreground color (default is black).

4. **display_info_airportsDB(text, fg='black')**:
    - Updates the `label_graph_airportsInfo` widget's text and foreground color.
    - Parameters:
        * `text`: Text to be displayed.
        * `fg`: Foreground color (default is black).

5. **display_info_frequenciesDB(text, fg='black')**:
    - Updates the `label_graph_frequenciesInfo` widget's text and foreground color.
    - Parameters:
        * `text`: Text to be displayed.
        * `fg`: Foreground color (default is black).

6. **display_info_runwaysDB(text, fg='black')**:
    - Updates the `label_graph_runwaysInfo` widget's text and foreground color.
    - Parameters:
        * `text`: Text to be displayed.
        * `fg`: Foreground color (default is black).

Each function aids in keeping the user informed about various operations, system status, or feedback within the application.


In [ ]:
""" functions Display Text """

# set of functions that set the text of the many infomation labels

def display_label_db(text, fg='black'):

    label_db['text'] = str(text)
    label_db['fg'] = fg
    label_db.update()


def display_label_graphs(text):
    labelFrame_graph['text'] = str(text)
    labelFrame_graph.update()


def display_info_bottom(text, fg='black'):

    # clear_graph_frame()
    label_graph_bottom['text'] = text
    label_graph_bottom['fg'] = fg
    label_graph_bottom.update()

def display_info_airportsDB(text, fg='black'):
    label_graph_airportsInfo['text'] = text
    label_graph_airportsInfo['fg'] = fg
    label_graph_airportsInfo.update()

def display_info_frequenciesDB(text, fg='black'):
    label_graph_frequenciesInfo['text'] = text
    label_graph_frequenciesInfo['fg'] = fg
    label_graph_frequenciesInfo.update()

def display_info_runwaysDB(text, fg='black'):
    label_graph_runwaysInfo['text'] = text
    label_graph_runwaysInfo['fg'] = fg
    label_graph_runwaysInfo.update()



# Functions for Database Interaction

The following are key functions that interact with various databases to check their status and load relevant data:

1. **check_db_status()**:
    - Checks the status of three collections: `collection_airports`, `collection_frequencies`, and `collection_runways`.
    - Updates global status variables (`airportsColl_status`, `frequenciesColl_status`, and `runwaysColl_status`) based on the presence of documents in the respective collections.
    - Uses helper functions to display the status of each collection on the GUI.

2. **load_db_data()**:
    - Attempts to load data from the database into the application at startup.
    - Initializes dataframes `df_airports` and `df_frequencies` for storing data.
    - Checks if both `collection_airports` and `collection_frequencies` have sufficient data before attempting to load. If not, informs the user to populate the database.
    - If both databases have data, retrieves information and stores it into pandas dataframes.
    - Upon successful data retrieval, updates the GUI to indicate data availability and then processes the loaded data with a `data_processing()` function.

### Key Points:

- **Global Variables**: Both functions use several global variables to track the status of different collections and to store the data.
- **Feedback Mechanism**: Throughout the database operations, various helper functions (`display_info_*`) are used to provide feedback to the user via the GUI. This ensures that the user is always informed about the status of the database and data availability.



In [ ]:
""" Functions Database """


# This function checks wether the databases relevalt for the calculations contains data.
# Status variables are set depending on each of the databases checks.
# This status variables are used withing another function in order to check if 
# data is ready to be downloaded from the database

def check_db_status():

    global airportsColl_status
    global frequenciesColl_status
    global runwaysColl_status

    if (collection_airports.count_documents({}) == 0):
        airportsColl_status = False
        display_info_airportsDB('Airports DataBase:\tEmpty','red')
    else: 
        airportsColl_status = True  
        display_info_airportsDB('Airports DataBase:\t\tOK','green')   

    if (collection_frequencies.count_documents({}) == 0):
        frequenciesColl_status = False
        display_info_frequenciesDB('Freqs DataBase:\tEmpty','red')
    else: 
        frequenciesColl_status = True
        display_info_frequenciesDB('Frequencies DataBase:\tOK','green') 

    if (collection_runways.count_documents({}) == 0):
        runwaysColl_status = False
        display_info_runwaysDB('Runways DataBase:\tEmpty','red')
        
    else: 
        runwaysColl_status = True
        display_info_runwaysDB('Runways DataBase:\t\tOK','green')

    
# this function attempts to load data from the datatabse as soon as the GUI is initiated.

def load_db_data():


    display_info_bottom('Updating application data...')
    time.sleep(1)

    global airportsColl_status
    global frequenciesColl_status
    global runwaysColl_status
    global df_airports
    global df_frequencies
    global enough_data_status

    # initialize dataframes 
    df_airports = pd.DataFrame()
    df_frequencies = pd.DataFrame()

    # we check here if the databases relevant for the calculations contain data 
    # before attemping to retrieve the information
    if (airportsColl_status == False) | (frequenciesColl_status == False):
        
        display_info_bottom('DataBase does not contain sufficient data. Check DB status and populate... ','red')
        enough_data_status = False

    else: 

        """ load data from mongo db """
        
        # if both datatbase contain data, we proceed to retreive the info and stoe it in pandas dataframes
        # we set a status variable that will be used in case user attempt to dispaly 
        # graphics without data being loaded onto memory.

        enough_data_status = True
        df_airports = pd.DataFrame(db.airports_collection.find({}))
        df_frequencies = pd.DataFrame(db.frequencies_collection.find({}))
        
        display_info_bottom('Data succesfully updated','green')
        time.sleep(1)
        display_label_graphs('Select Statistics to Display')
        display_info_bottom('')

        # after the datatframes have been set, we call the function that cleans and manioulates the data.
        data_processing()

    

## Dataframe Processing

The primary function detailed here, **data_processing()**, is responsible for the data wrangling and manipulation of pandas dataframes. This function is critical in preparing the data for subsequent visual and analytical operations. 

### Details:

- **Data Cleanup**:
    - Drops duplicate records from the `df_frequencies` and `df_airports` dataframes.
    - Filters relevant columns for frequencies and renames columns for clarity.
    
- **Airport DataFrame Manipulation**:
    - Focuses on the airports located in Great Britain (`iso_country` column value is 'GB').
    - Filters for `small_airport`, `medium_airport`, and `large_airport` types.
    - Creates three new columns to represent each airport size type.
    - Renames the `type` column to `size type` for clarity.
    
- **Merging DataFrames**:
    - The `air_df` and `freq_df` dataframes are merged on the 'ident' and 'airport_ident' columns. 
    - Post-merging, the frequency value is set for each size column (small, medium, and large).
    - Columns are renamed and reorganized for uniformity.
    
- **Data Consistency Fixes**:
    - Addresses potential inconsistencies in the `freq type` column by replacing varying entries with a standardized format.

- **Dataframe Type Conversions**:
    - Converts certain columns from string data type to float for analytical operations.

- **Dataframe Segregation**:
    - Based on the airport's size type, the data is divided into `small_df`, `medium_df`, and `large_df` dataframes. Each dataframe focuses solely on one airport size type.
    
- **Pie Chart Data Preparation**:
    - Groups the `merged_df` by `freq type` for `small_airport` size type.
    - Counts the occurrences of each frequency type and sorts them in descending order. This data is stored in the `small_to_graph` dataframe and will likely be used for visual representation.

### Key Points:

- **Efficiency**: The function efficiently manages and manipulates multiple dataframes in a step-by-step manner, ensuring that the data is in the right shape for visualization and analysis.
- **Flexibility**: By breaking down data into multiple dataframes and addressing inconsistencies, the function ensures accurate and comprehensive analyses can be conducted.



In [ ]:
""" dataframes processing functions """

# This function is where all the  wrangling happens.
# We manipulate the dataframes and set them ready for the functions that take care of the output.

def data_processing():

    global merged_df
    global small_df
    global medium_df
    global large_df
    global small_to_graph
    

    # df drop duplicates: 
    freq_df = df_frequencies.drop_duplicates()
    airport_df = df_airports.drop_duplicates()
    
    # filtering columns
    freq_df = freq_df[['airport_ident', 'type', 'description','frequency_mhz']]
    # rename freq type column
    freq_df = freq_df.rename(columns={'type': 'frequency type'})

    # airports df GB only
    air_df_gb = airport_df[airport_df['iso_country']=='GB']
    air_df_gb 

    # filter columns and keep only relevant ones:
    air_df_gb = air_df_gb[['ident','name','type','iso_country']]
    air_filter_list = ['small_airport','medium_airport','large_airport']
    air_df = air_df_gb[air_df_gb['type'].isin(air_filter_list)]

    # airports df new type columns
    air_df.loc[air_df['type']=='small_airport', 'small airport'] = 1
    air_df.loc[air_df['type']=='medium_airport', 'medium airport'] = 1
    air_df.loc[air_df['type']=='large_airport', 'large airport'] = 1


    # rename airport type column:
    air_df = air_df.rename(columns={'type': 'size type'})
    air_df

    # merge dataframes
    merged_df = air_df.merge(freq_df, how='left' , left_on='ident', right_on='airport_ident' )

    # pass the freq value to size columns
    merged_df.loc[merged_df['small airport']==1, 'small airport'] = merged_df['frequency_mhz']
    merged_df.loc[merged_df['medium airport']==1, 'medium airport'] = merged_df['frequency_mhz']
    merged_df.loc[merged_df['large airport']==1, 'large airport'] = merged_df['frequency_mhz']

    # list of new names for columns
    column_name_list = [ 'ident', 'name', 'size type', 'iso country', 'small airport',
                        'medium airport', 'large airport', 'airport ident', 'freq type',
                        'description', 'frequency mhz']

    # set new colum names
    merged_df.columns = column_name_list

    # drop nan values using freq type column as guide
    # (Entries with no freq data are not relevant for calculations.)
    merged_df = merged_df.dropna(subset=['frequency mhz'])

    # organize name inconsistencies in column 'freq type'
    merged_df = merged_df.replace([ 'Safety com', 'Safetycom', 'SafetyCom','safetycom', 
                                    'Safety Comm','SAFETYCOM'], 'SAFE/COM')
    merged_df = merged_df.replace([ 'Dir', 'Director'] , 'DIR')
    merged_df = merged_df.replace([ 'Info'] , 'INFO')
    merged_df = merged_df.replace([ 'Ops'] , 'OPS')
    merged_df = merged_df.replace([ 'grn','GRN'] , 'GND')
    merged_df = merged_df.replace([ 'Tower'] , 'TWR')
    merged_df = merged_df.replace([ 'APP/RAD'] , 'RAD/APP')
    merged_df = merged_df.replace([ 'Fire service'] , 'FIRE')
    merged_df = merged_df.replace([ 'App', 'APP vhf'], 'APP')
    merged_df = merged_df.replace([ 'Radar', 'RDR'], 'RADAR')

    # convert size type columns string values into float
    merged_df['large airport'] = merged_df['large airport'].astype(float)
    merged_df['medium airport'] = merged_df['medium airport'].astype(float)
    merged_df['small airport'] = merged_df['small airport'].astype(float)
    merged_df['frequency mhz'] = merged_df['frequency mhz'].astype(float)

    corr_df = merged_df

    # separating into individual dfs for easier display:
    small_df = corr_df[corr_df['size type']=='small_airport']
    small_df = small_df[['size type','frequency mhz']]

    medium_df = corr_df[corr_df['size type']=='medium_airport']
    medium_df = medium_df[['size type','frequency mhz']]

    large_df = corr_df[corr_df['size type']=='large_airport']
    large_df = large_df[['size type','frequency mhz']]


    # arrange data for pie chart
    merged_df_small = merged_df[merged_df['size type']=='small_airport']
    grouped_df_small = merged_df_small.groupby(['freq type','size type'])
    small_to_graph = grouped_df_small['frequency mhz'].count()
    small_to_graph = small_to_graph.droplevel(1).sort_values(ascending=False)



# Graphing Utilities
## Functions for Graph Generation

This suite of functions is primarily used for generating various graphs based on user input:

1. **`large_airport_hist_graph()`**:
    - Connected to button number one on the stats tab.
    - Displays a histogram based on the selected radio button option.
        * If `value == 1`: Shows histogram for large airports frequencies.
        * If `value == 2`: Showcases frequencies greater than 100MHz across all airport sizes.

2. **`small_airport_comm_graph()`**:
    - Connected to button number two on the stats tab.
    - Depending on the radio button selection, it will:
        * Display communication frequencies by type for small airports either as a pie chart or a bar chart.

3. **`correlation_graph()`**:
    - Connected to button number three on the stats tab.
    - Provides a correlation heatmap.
        * Checks user's choice for interval scaling (either 1MHz or 5MHz).
        * Shows correlation statistics between frequencies of small, medium, and large airports.

4. **`clear_graph_frame()`**:
    - A utility function.
    - Clears the contents of the `labelFrame_graph`.
        * Iterates through each child widget of the frame and destroys it.

5. **`make_corr_text()`**:
    - Creates labels within the `labelFrame_graph` frame.
    - Displays correlation values between different airport sizes.

6. **`make_stats_text()`**:
    - Generates labels within the `labelFrame_graph` frame.
    - Showcases Mean, Median, and Mode statistics.

The functionalities provided by these functions ensure that the user can visualize the data in an insightful manner, catering to varied analytical requirements.


In [ ]:
""" graphs functions """

# this function is attched to button number one on stats tab
def large_airport_hist_graph():

    # we retrieve value from radio button selection
    value = var_stats1.get()

    if (value == 1) | (value == 2): # check if there is a selection made:

        if enough_data_status: # chekc if there is data from database in order to make calculations

            if value == 1 :

                clear_graph_frame()
                display_label_graphs('Loading data...')

                fig = plt.Figure(figsize=(6,4))
                ax = fig.add_subplot(1, 1, 1)
                ax.set_xlabel("Frequency (MHz)")
                ax.set_title('Large Airports Frequencies Histogram')
                bar1 = FigureCanvasTkAgg(fig, labelFrame_graph)
                bar1.get_tk_widget().place(relx=0.02, rely=0.08)

                n_bins = np.arange(0,200,1)

                df1 = merged_df['large airport']
                df1.plot(kind='hist', ax=ax, legend=True, bins= n_bins, color='red')

                make_stats_text()

                txt1 = 'Mean    :  ' + str( round(merged_df['large airport'].mean(), 2) )
                txt2 = 'Median :  ' + str( round(merged_df['large airport'].median(), 2) )
                txt3 = 'Mode    :  ' + str( list(merged_df['large airport'].mode()) )
                label_graph_mean['text'] = txt1
                label_graph_median['text'] = txt2
                label_graph_mode['text'] = txt3

                # set disply tittle
                display_label_graphs('Mean | Median | Mode --- Large Airports')

            elif value == 2 :
                

                clear_graph_frame()
                display_label_graphs('Loading data...')

                fig = plt.Figure(figsize=(6,4))
                ax = fig.add_subplot(1, 1, 1)
                ax.set_xlabel("Frequency (MHz)")
                ax.set_title('Frequencies Greater Than 100Mhz Across Airport Sizes')
                bar1 = FigureCanvasTkAgg(fig, labelFrame_graph)
                bar1.get_tk_widget().place(relx=0.02, rely=0.08)

                n_bins = np.arange(100,500,1)

                df1 = merged_df.loc[merged_df['frequency mhz']>100]

                df1[['frequency mhz']].plot(kind='hist', ax=ax, legend=True, bins= n_bins, color='black')

                make_stats_text()

                filtered_morethan100 = merged_df.loc[merged_df['frequency mhz']>100]
                # filtered_morethan100['frequency mhz'].agg(['mean','median'])
                filtered_morethan100['frequency mhz'].mode()
                txt1 = 'Mean    :  ' + str( round(filtered_morethan100['frequency mhz'].mean(), 2) )
                txt2 = 'Median :  ' + str( round(filtered_morethan100['frequency mhz'].median(), 2) )
                txt3 = 'Mode    :  ' + str( round(float(filtered_morethan100['frequency mhz'].mode()), 2) )
                label_graph_mean['text'] = txt1
                label_graph_median['text'] = txt2
                label_graph_mode['text'] = txt3

                # set display tittle
                display_label_graphs('Mean | Median | Mode --- Frequencies Greater Than 100 Mhz Across Sizes')

        else: 

            display_label_graphs('Data Not Loaded')
            display_info_bottom('DataBase does not contain sufficient data. Check DB status and populate... ',
                                'red')

    else: display_label_graphs('Must select an option before generating Mean|Median|Mode Stats')

# function attache dto button 2 stats tab
def small_airport_comm_graph():

    # retreive value from radiobutton selection
    value = var_comm1.get()

    if (value == 1) | (value == 2): # check if there is a selection made:

        if enough_data_status: # check if database data has been retreived

            if value == 1:

                clear_graph_frame()
                display_label_graphs('Loading data...')

                fig = plt.Figure(figsize=(8,5))
                ax = fig.add_subplot(1, 1, 1)
      
                bar1 = FigureCanvasTkAgg(fig, labelFrame_graph)
                bar1.get_tk_widget().place(relx=0.2, rely=0.02)

                cs=cm.Set1(np.arange(40)/7)

                df1 = small_to_graph

                df1.plot(kind='pie', ax=ax, rotatelabels=True, colors=cs, label = '' ,
                                    autopct=lambda p: format(p, '.2f')if p > 4 else None )

                display_label_graphs('Small Airports Communication Frequencies by Type | Pie Chart')




            elif value == 2:

                clear_graph_frame()
                display_label_graphs('Loading data...')

                fig = plt.Figure(figsize=(8,5))
                ax = fig.add_subplot(1, 1, 1)
                # ax.set_xlabel("Frequency (MHz)")
                ax.set_ylabel('Counts')
                # ax.set_title('Small Airports Communication Frequencies')
                bar1 = FigureCanvasTkAgg(fig, labelFrame_graph)
                bar1.get_tk_widget().place(relx=0.2, rely=0.02)


                cs=cm.Set1(np.arange(40)/7)

                df1 = small_to_graph
                # df1 = merged_df.groupby(['small airport'])
                # plt.legend(loc="left")
                df1.plot(kind='bar', ax=ax, color=cs ) 

                string_pie = 'Top 5 frequencies by type:\n'  
   
                for i in range(5):
                    a = small_to_graph.index[i]
                    b = small_to_graph[i]
                    string_pie = string_pie +'{:-<10s}{:>10}\n'.format(a,b)
                

                display_label_graphs('Small Airports Communication Frequencies by Type | Bar Chart')

        else: 

            display_label_graphs('Data Not Loaded')
            display_info_bottom('DataBase does not contain sufficient data. Check DB status and populate... ','red')
    
    else: display_label_graphs('Must select an option before generating communication frequencies statistics')

# function attached to button 3 stats tab
def correlation_graph():

    # retrieve value from radiobuttons selction
    value = var_corr1.get()


    if (value == 1) | (value == 2): # check if there is a selection made:

        if enough_data_status: # check if database data has been retreived

            clear_graph_frame()

            # set the intervals scale based on radiobutton choice (set variable n_bins)
            if value == 1:
                n_bins = np.arange(0,500,1)
                display_label_graphs('Loading data... ')

            else: 
                n_bins = np.arange(0,500,5)
                display_label_graphs('Loading data... ')
                
            if enough_data_status: # chekc if there is data from database in order to make calculations

                fig2, ax = plt.subplots(1, 3, figsize=(10,2))
                graph2 = FigureCanvasTkAgg(fig2, labelFrame_graph)
                graph2.get_tk_widget().place(relx=0.02, rely=0.06)

                n_small, _, _ = ax[0].hist(small_df["frequency mhz"], bins=n_bins, color='blue')
                n_medium, _, _ = ax[1].hist(medium_df["frequency mhz"], bins=n_bins, color='green')
                n_large, _, _ = ax[2].hist(large_df["frequency mhz"], bins=n_bins, color='red')

                ax[0].set_xlabel("Frequency (MHz)")
                ax[0].set_title('Small Airports Frequencies')

                ax[1].set_xlabel("Frequency (MHz)")
                ax[1].set_title('Medium Airports Frequencies')

                ax[2].set_xlabel("Frequency (MHz)")
                ax[2].set_title('Large Airports Frequencies')
                

                # create data for the calculation of correlations by merging these dataframes
                # Note that these have the same number of rows with the same bins.
                dataFrameCorr = {'Sml': n_small, 'Med': n_medium, 'Lrg': n_large}
                # DataFrame
                df = pd.DataFrame(dataFrameCorr)       

                # Matrix
                corrMatrix = df.corr()
    
                fig3, ax3 = plt.subplots(1, 1, figsize=(3.5,2.5))
                graph3 = FigureCanvasTkAgg(fig3, labelFrame_graph)
                graph3.get_tk_widget().place(relx=0.2, rely=0.5)
                ax3.set_xlabel('Heat Map')
                ax3 = sns.heatmap( corrMatrix , linewidth = 0.5, vmin=0 , annot= True, cmap = 'coolwarm' )

                make_corr_text()  # create the textlabels
                corrMatrix = round(corrMatrix,2)  #round decimal.

                text1 = 'Correlation Small/Medium:\t' + str(round((np.corrcoef(n_small, n_medium)[0, 1]),2))
                label_graph_corr1['text'] = text1
                text2 = 'Correlation Small/Large:\t' + str(round((np.corrcoef(n_small, n_large)[0, 1]),2))
                label_graph_corr2['text'] = text2
                text3 = 'Correlation Medium/Large:\t' + str(round((np.corrcoef(n_medium, n_large)[0, 1]),2))
                label_graph_corr3['text'] = text3

                if value ==1:
                    display_label_graphs('Correlations Between Frequencies | 1 Mhz Intervals ')
                elif value==2: display_label_graphs('Correlations Between Frequencies | 5 Mhz Intervals ')

        else:

            display_label_graphs('Data Not Loaded')
            display_info_bottom('DataBase does not contain sufficient data. Check DB status and populate... ',
                                'red')    

    else: display_label_graphs('Must select an option before generating correlation statistics')


def clear_graph_frame():

    for widget in labelFrame_graph.winfo_children():
        widget.destroy()

# function that creates labels for the correlation information
def make_corr_text():

    global label_graph_corr1
    global label_graph_corr2
    global label_graph_corr3

    """ M/M/M Stats Text Info """
    x_coor_corr = 0.5
    y_coor_corr = 0.6
    gap_coor_crr = 0.08
  
    label_graph_corr1 =  tk.Label(labelFrame_graph, text='', font=('arial', 20)) 
    label_graph_corr1.place(relx=x_coor_corr, rely=y_coor_corr)

    label_graph_corr2 =  tk.Label(labelFrame_graph, text='', font=('arial', 20)) 
    label_graph_corr2.place(relx=x_coor_corr, rely=y_coor_corr + gap_coor_crr)

    label_graph_corr3 =  tk.Label(labelFrame_graph, text='', font=('arial', 20)) 
    label_graph_corr3.place(relx=x_coor_corr, rely=y_coor_corr + gap_coor_crr*2)


# function that creates labels for the mean/median/mode information
def make_stats_text():

    global label_graph_mean
    global label_graph_median
    global label_graph_mode

    """ M/M/M Stats Text Info """
    x_coor_stats = 0.55
    y_coor_stats = 0.1
    y_gap_stats = 0.08

    # Info label at the bottom
    label_graph_mean =  tk.Label(labelFrame_graph, text='', font=('arial', 20)) 
    label_graph_mean.place(relx=x_coor_stats, rely=y_coor_stats)


    # Info label at the bottom
    label_graph_median =  tk.Label(labelFrame_graph, text='', font=('arial', 20)) 
    label_graph_median.place(relx=x_coor_stats, rely=y_coor_stats + y_gap_stats)


    # Info label at the bottom
    label_graph_mode =  tk.Label(labelFrame_graph, text='', font=('arial', 20)) 
    label_graph_mode.place(relx=x_coor_stats, rely=y_coor_stats + y_gap_stats*2)


# UI

## Root Window Initialization
### Setting up the Main Application Window

This section of the code initiates the primary application window using the `tkinter` library:

This initialization ensures that the main application window is appropriately sized and titled for its intended use: displaying airport frequencies.


In [ ]:
""" root window """

root = tk.Tk()
root_WIDTH = 800
root_HEIGHT = 600
root.geometry('1300x800')
root.title('Airport Fequencies')
root.minsize(root_WIDTH,root_HEIGHT)


## Notebook and Frame Initialization
### Setting up Tabs using Notebook Widget

This section of the code sets up a tabbed interface using the `Notebook` widget from the `ttk` library:

Through this setup, the application now has a tabbed interface allowing users to switch between different content areas, namely 'Statistics & Graphs' and 'Database'.


In [ ]:
""" notebook """


# create a notebook
notebook = ttk.Notebook(root)
notebook.pack(pady=0, expand=False)


# create frames
frame_WITDH = 1400
frame_HEIGHT = 800
frame1 = ttk.Frame(notebook, width=frame_WITDH, height=frame_HEIGHT)
frame2 = ttk.Frame(notebook, width=frame_WITDH, height=frame_HEIGHT)

frame1.pack(fill='both', expand=False)
frame2.pack(fill='both', expand=False)


# add frames to notebook: Note that the order of the tabs depends on which line runs first
notebook.add(frame2, text='Statistics & Graphs')
notebook.add(frame1, text='Database')



## Tab 1: Frame | Statistics & Graphs

### Frames Initialization

This portion of the code creates and positions several frames (grouping widgets) to organize the content in the 'Statistics & Graphs' tab:

### Text Labels for Database Status

Three text labels provide information about the status of different sections of the database:

### Radio Buttons

Radio buttons allow the user to select specific options. They are grouped based on their functionality:

### Action Buttons

Buttons trigger various functionalities:



In [ ]:
""" Tab: Frame 2 | Statistics & Graphs """

""" frames """
# frame for graphics display
labelFrame_graph = tk.LabelFrame(frame2,text='', font=('arial', 25), height=560, width=1220, border=1)
labelFrame_graph.place(x=10, y=160)


# Database status frame:
labelFrame_dbStatus = tk.LabelFrame(frame2,text='DataBase Status: ', 
                                    font=('arial', 12), height=150, width=295, border=1)
labelFrame_dbStatus.place(relx=0.75, rely=0.01)


# Correlations frame:
labelFrame_correlations = tk.LabelFrame(frame2,text='Correlation Stats: ', 
                                    font=('arial', 12), height=150, width=295, border=1)
labelFrame_correlations.place(relx=0.505, rely=0.01)

# Small airports stats frame:
labelFrame_smallAirports = tk.LabelFrame(frame2,text='Small Airports Communication Frequencies: ', 
                                            font=('arial', 12), height=150, width=295, border=1)
labelFrame_smallAirports.place(relx=0.255, rely=0.01)

# Mean|Median|Mode stats frame:
labelFrame_meanMedianMode = tk.LabelFrame(frame2,text='Mean|Median|Mode Stats: ', 
                                            font=('arial', 12), height=150, width=295, border=1)
labelFrame_meanMedianMode.place(x=10, rely=0.01)

""" text labels Database Status """
# Info airport db info:
label_graph_airportsInfo =  tk.Label(labelFrame_dbStatus, text='', font=('arial', 15), fg='green') 
label_graph_airportsInfo.place(relx=0.1, rely=0.1)
# Info frequencies db info:
label_graph_frequenciesInfo =  tk.Label(labelFrame_dbStatus, text='', font=('arial', 15), fg='green') 
label_graph_frequenciesInfo.place(relx=0.1, rely=0.25)
# Info runways db info:
label_graph_runwaysInfo =  tk.Label(labelFrame_dbStatus, text='', font=('arial', 15), fg='green') 
label_graph_runwaysInfo.place(relx=0.1, rely=0.40)



# Info label at the bottom
label_graph_bottom =  tk.Label(labelFrame_graph, text='', font=('arial', 25), fg='green') 
label_graph_bottom.place(relx=0.01, rely=0.92)


""" Radio Buttons """

x_coord_rad = 0.1
y_coord_rad = 42
rad_btn_txt_W = 16
rad_btn_sep = 23

# large airports mean/median/mode
var_stats1 = IntVar()

radio_button1 = Radiobutton(labelFrame_meanMedianMode, text=' Large Airports', width=rad_btn_txt_W, 
                            anchor='w' , variable=var_stats1, value=1)
radio_button2 = Radiobutton(labelFrame_meanMedianMode, text=' Freqs > 100[Mhz]', width=rad_btn_txt_W, 
                            anchor='w' , variable=var_stats1, value=2)

radio_button1.place(relx=x_coord_rad, rely=0.15)
radio_button2.place(relx=x_coord_rad, rely=0.35)

# Correlations:
var_corr1 = IntVar()

radio_button1_corr = Radiobutton(labelFrame_correlations, text=' 1Mhz Intervals', width=rad_btn_txt_W, 
                            anchor='w' , variable=var_corr1, value=1)
radio_button2_corr  = Radiobutton(labelFrame_correlations, text=' 5Mhz Intervals', width=rad_btn_txt_W, 
                            anchor='w' , variable=var_corr1, value=2)

radio_button1_corr.place(relx=x_coord_rad, rely=0.15)
radio_button2_corr.place(relx=x_coord_rad, rely=0.35)

# Small Comm Freqs:
var_comm1 = IntVar()

radio_button1_comm = Radiobutton(labelFrame_smallAirports, text=' Pie Chart', width=rad_btn_txt_W, 
                            anchor='w' , variable=var_comm1, value=1)
radio_button2_comm  = Radiobutton(labelFrame_smallAirports, text=' Bar Chart', width=rad_btn_txt_W, 
                            anchor='w' , variable=var_comm1, value=2)

radio_button1_comm.place(relx=x_coord_rad, rely=0.15)
radio_button2_comm.place(relx=x_coord_rad, rely=0.35)



""" Buttons """

# coordinates:
x_coord = 40
y_coord = 40
y_gap = 60
x_gap = 60
btn_h = 4
btn_w = 10

# buttons rel coordinates:
x_coord_frame2 = 0.25
y_coord_frame2 = 0.7


# button generate stats (mean/median/mode)
button_stats1 = tk.Button(labelFrame_meanMedianMode, text='Generate', width=btn_w, 
                                command=large_airport_hist_graph).place( relx=x_coord_frame2 , rely=y_coord_frame2)

# button generate stats (Airports Graph)
button_smallAirport = tk.Button(labelFrame_smallAirports, text='Generate', width=btn_w, 
                                command=small_airport_comm_graph).place( relx=x_coord_frame2 , rely=y_coord_frame2)

# button correlation                           
button_correlation = tk.Button(labelFrame_correlations, text='Generate', width=btn_w, 
                                command=correlation_graph).place( relx=x_coord_frame2 , rely=y_coord_frame2)






## Tab 2: Frame | Database

### Preview Frame

The code sets up a frame called `labelFrame_tree` for previewing database content:

### Treeview

A treeview widget, `my_tree`, is used to provide a tabular view of the data:

### Frames for Buttons

Two distinct frames are used to house buttons related to database operations and CSV file interactions:

### Labels

A label to display progress or status related to database actions:

### Action Buttons

There are two primary action buttons allowing for CSV file loading and database manipulation:


In [ ]:
""" Tab: Frame | Database """


#********************************************************************************************************
""" preview frame """

labelFrame_tree = tk.LabelFrame(frame1,text='Preview', font=('arial', 25) , height=560, width=1220, border=1)
labelFrame_tree.place(x=10, y=160)



""" treeview """
my_tree = ttk.Treeview(labelFrame_tree, height=28)

""" frames """
# frame for DB button area
label_button_db = tk.LabelFrame(frame1, text='DataBase', height=150, width=605, border=1)
label_button_db.place(x=625, rely=0.01)

# frame for DB button area
label_button_csv = tk.LabelFrame(frame1, text='CSV File', height=150, width=605, border=1)
label_button_csv.place(x=10, rely=0.01)


""" labels """
# DB progess label text:
label_db =  tk.Label(label_button_db, text='', fg='green') 
label_db.place(x=40, y=70)


""" buttons """
button_frame1_csv = tk.Button(label_button_csv, text='Load CSV', 
                                width=btn_w, command=read_csv).place( x=x_coord , y=y_coord)
                                
button_frame1_db = tk.Button(label_button_db, text='Add to Database',
                                width=btn_w, command=to_mongoDB ).place( x=x_coord , y=y_coord)




#********************************************************************************************************


# Main Loop


In [ ]:
""" Main Loop | Tail """
# global file_loaded_status
file_loaded_status = False

# execution of these functions happen everytime user open the program.
# database is checked and data is loaded into program's memory.
check_db_status()

load_db_data()

root.mainloop()